In [1]:
import os #, logging, argparse
import dota2api
# from util import get_game_mode_string
from pymongo import MongoClient
from time import sleep
from sys import exit

In [3]:
#connect to mongo db
mongo_client=MongoClient()
db = mongo_client.dota2
coll = db.match_history


In [4]:
#set up dota2api

api = dota2api.Initialise(logging=True)

In [6]:
start_match_seq_num = 1704401237

In [7]:
match_temp = api.get_match_history_by_seq_num(start_match_seq_num-1)

In [9]:
match_temp.keys()

[u'status', u'matches']

In [12]:
match_temp['matches'][0]['match_seq_num']

1704401237

In [13]:
len(match_temp)

2

In [14]:
coll.insert(match_temp)

ObjectId('567270ae22f14a0f2ec02f29')

In [16]:
coll.count()

1

In [19]:
match_temp['status']

1

** How to deal with the status not '1'

In [20]:
type(match_temp)

dota2api.src.response.Dota2Dict

In [21]:
type(match_temp['matches'])

list

In [22]:
type(match_temp['status'])

int

In [23]:
len(match_temp['matches'])

100

In [24]:
db.coll.remove()

{u'n': 0, u'ok': 1}

In [26]:
if match_temp['status'] != 1:
    print match_temp['statusDetail']
else:
    for match in match_temp['matches']:
        db.coll.insert(match)

In [27]:
db.coll.count()

100

In [29]:
import datetime

In [30]:
t1 = datetime.datetime.now()

In [31]:
t1

datetime.datetime(2015, 12, 17, 0, 41, 38, 348199)

In [32]:
t2 = datetime.datetime.now()

In [33]:
t2

datetime.datetime(2015, 12, 17, 0, 41, 56, 733514)

In [34]:
t2 - t1

datetime.timedelta(0, 18, 385315)

In [35]:
print str(t2-t1)

0:00:18.385315


In [ ]:
def get_match_history():
    n=0
    start_match_seq_num = 1704401237
    t0 = datetime.datetime.now()
    
    while n < 400:
        if n % 40 ==0:
            num = n+1
            t1 = datetime.datetime.now()
            delta_t = str(t1 - t0)
            print "Process {0} request use time: {1}".format(num, delta_t)
            
        sleep(1.0)
        
        gmh = api.get_match_history_by_seq_num(start_at_match_seq_num=start_match_seq_num,
                                               matches_requested=400)
        
        if gmh['status'] != 1:
            print gmh['statusDetail']
        else:
            for match in gmh['matches']:
                
                match_id = match['match_id']

                if db.coll.find_one({'match_id':match_id}) != None:
#                 logger.debug('Encountered match %s already in database, exiting.' % match_id)
                    print '.'
                    exit(0)

                sleep(1.0)
                
                db.coll.insert(match)
                
        last_match_seq_num = match[-1]['match_seq_num']
        logger.debug('Match_id of last match of GMH query: %s' % last_match_id)
        # We don't want to record the last match twice, so subtract 1
        start_match_seq_num = last_match_seq_num + 1
        n+=1

In [37]:
db.coll.count()

200

In [39]:
match_temp['matches'][0]['match_id']

1928833457

In [41]:
db.coll.find_one({'match_id':1928833457})

{u'_id': ObjectId('567273e722f14a0f2ec02f2a'),
 u'barracks_status_dire': 51,
 u'barracks_status_radiant': 63,
 u'cluster': 133,
 u'duration': 1691,
 u'engine': 1,
 u'first_blood_time': 109,
 u'game_mode': 22,
 u'human_players': 10,
 u'leagueid': 0,
 u'lobby_type': 7,
 u'match_id': 1928833457,
 u'match_seq_num': 1704401237,
 u'negative_votes': 32,
 u'players': [{u'ability_upgrades': [{u'ability': 5163,
     u'level': 1,
     u'time': 475},
    {u'ability': 5164, u'level': 2, u'time': 575},
    {u'ability': 5163, u'level': 3, u'time': 585},
    {u'ability': 5162, u'level': 4, u'time': 722},
    {u'ability': 5163, u'level': 5, u'time': 806},
    {u'ability': 5165, u'level': 6, u'time': 1101},
    {u'ability': 5163, u'level': 7, u'time': 1190},
    {u'ability': 5162, u'level': 8, u'time': 1288},
    {u'ability': 5162, u'level': 9, u'time': 1466},
    {u'ability': 5162, u'level': 10, u'time': 1673},
    {u'ability': 5165, u'level': 11, u'time': 1708},
    {u'ability': 5164, u'level': 12, u'

* first two hundreds records are dupblicated

In [49]:
db.coll.count()

1201